# Preprocessing from package

## Imports packages and sample .txt

Checking virtualenv with `pyenv`

In [1]:
#!pyenv virtualenvs

Importing necessary packages

In [2]:
#!pip install nltk

In [11]:
import os
import pandas as pd
from IIPE.preproc import make_contents_df, make_tokens
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Select the data_samples to read .txt files

In [4]:
print(os.getcwd())
os.chdir(os.path.join('..', 'IIPE', 'data_sample', 'plain_text_sample'))
print(os.getcwd())

/Users/alexisgourdol/code/alexisgourdol/IIPE-data/notebooks
/Users/alexisgourdol/code/alexisgourdol/IIPE-data/IIPE/data_sample/plain_text_sample


In [5]:
file_names = os.listdir()
file_names

['Reports_Plain text_03220F_15_11_2019.txt',
 'Reports_Plain text_01300Q_08_10_2020.txt',
 'Results\\Word count\\Whole sample.png',
 'Reports_Plain text_03917V_23_09_2020.txt',
 'Reports_Plain text_05933G_08_10_2020.txt',
 'Reports_Plain text_07518E_15_12_2020.txt']

## Use our preprocessing functions

In [6]:
df_contents = make_contents_df(file_names)
df_contents

,date,reference,text
0,2019-11-15,03220F,"Whole-School Evaluation – Management, Leadersh..."
1,2020-10-08,01300Q,"Whole-School Evaluation – Management, Leadersh..."
2,2020-09-23,03917V,"Whole-School Evaluation – Management, Leadersh..."
3,2020-10-08,05933G,"Whole-School Evaluation – Management, Leadersh..."
4,2020-12-15,07518E,"Whole-School Evaluation – Management, Leadersh..."


In [7]:
tokens = make_tokens(df_contents)
print(len(tokens), ' tokens available. Here are the 5 first in no particular order: ')
tokens[:5]

4040  tokens available. Here are the 5 first in no particular order: 


['evaluation', 'leadership', 'date', 'inspection', 'inspection']

In [16]:
from nltk.corpus import stopwords

In [17]:
stopwords

<WordListCorpusReader in '/Users/alexisgourdol/nltk_data/corpora/stopwords'>

In [15]:
from IIPE.constants import NEW_STOP_WORDS
NEW_STOP_WORDS

['school',
 'learning',
 'student',
 'pupil',
 'teacher',
 'management',
 'teaching',
 'support',
 'lesson',
 'board',
 'schools',
 'learnings',
 'students',
 'pupils',
 'teachers',
 'managements',
 'teachings',
 'supports',
 'lessons',
 'boards']

In [ ]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [14]:
vectorizer = TfidfVectorizer().fit(df_contents['text'])

data_vectorized = vectorizer.transform(df_contents['text'])

lda_model = LatentDirichletAllocation(n_components=2).fit(data_vectorized)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('the', 0.5043428442938909), ('of', 0.504323311724729), ('and', 0.5043172294074748), ('to', 0.5042461398210671), ('in', 0.5042319912427571), ('school', 0.5042119078837702), ('pupils', 0.5041908716025133), ('is', 0.5041529632892041), ('learning', 0.5040994144416463), ('for', 0.503890790000769)]
Topic 1:
[('the', 2.870075533486435), ('of', 2.6081406807023635), ('and', 2.4265476274436777), ('to', 1.7005288775553065), ('in', 1.6550595143358604), ('school', 1.5144525730236702), ('pupils', 1.461736068267673), ('is', 1.378212172930212), ('learning', 1.239605963842298), ('for', 0.9971525833026216)]


## Topic modelling